In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import pandas as pd
import sklearn
import requests
import matplotlib.pyplot as plt

# Getting Data

## OHLC

In [2]:
def trade_bucket(startDate='2019-05-01'):
    start = 0
    df = pd.DataFrame()
    data = True
    counter = 1
    
    while data:
        api_call_url = 'https://www.bitmex.com/api/v1/trade/bucketed?binSize=1h&partial=false&symbol=XBTUSD&count=1000&start=' + str(
                       start) +'&reverse=true&startTime=' + startDate
        r = requests.get(api_call_url)
        if r.status_code != 200:
            r.raise_for_status()
        data = r.json()
        if data:
            temp_df = pd.DataFrame(data)
            df = df.append(temp_df)
            start += 1000
        if counter < 30:
            counter += 1
        else:
            sleep(60)
            counter = 1
    df['timestamp'] = pd.to_datetime(df['timestamp'], dayfirst=True)
    df.set_index('timestamp', inplace=True)
    return df

In [3]:
trade_df = trade_bucket()

## Funding Rates

In [35]:
funding_rates_df = pd.read_csv('data/funding.csv', parse_dates=True)
funding_rates_df['timestamp'] = pd.to_datetime(funding_rates_df['timestamp'], dayfirst=False)
funding_rates_df.index = funding_rates_df['timestamp']

## OV

In [36]:
OI_OV_df = pd.read_csv('data/OI_OV_XBT.csv', parse_dates=True)
OI_OV_df['timestamp'] = pd.to_datetime(OI_OV_df['timestamp'], dayfirst=False, utc=True)
OI_OV_df.index = OI_OV_df['timestamp']

## X, y

In [37]:
OI_OV_df['openInterest'].loc['2019-05-09 04:00:00+00:00':'2020-01-23 12:00:00+00:00':8]

timestamp
2019-05-09 04:00:00+00:00           591356894.3
2019-05-09 12:00:00+00:00           588270690.0
2019-05-09 20:00:00+00:00           607830646.5
2019-05-10 04:00:00+00:00           642309054.7
2019-05-10 12:00:00+00:00           684630780.9
                                       ...     
2020-01-22 03:59:55.966000+00:00    836009945.0
2020-01-22 11:59:58.714000+00:00    839956384.0
2020-01-22 19:59:57.178000+00:00    831338723.0
2020-01-23 03:59:59.056000+00:00    841315126.0
2020-01-23 11:59:59.579000+00:00    855594138.0
Name: openInterest, Length: 779, dtype: float64

In [38]:
a = funding_rates_df['fundingRate'].loc['2019-05-09 04:00:00+00:00':'2020-01-23 12:00:00+00:00'].pct_change().dropna().values.reshape(-1, 1)
b = OI_OV_df['openInterest'].loc['2019-05-09 04:00:00+00:00':'2020-01-23 12:00:00+00:00':8].pct_change().dropna().values.reshape(-1, 1)
c = OI_OV_df['openValue'].loc['2019-05-09 04:00:00+00:00':'2020-01-23 12:00:00+00:00':8].pct_change().dropna().values.reshape(-1, 1)
a.shape, b.shape, c.shape

((778, 1), (778, 1), (778, 1))

In [39]:
X = np.hstack((a,b,c))
y = trade_df['close'][::-1].loc['2019-05-09 04:00:00+00:00':'2020-01-23 12:00:00+00:00':8].pct_change().dropna().values.reshape(-1, 1)

## Random Forest Regressor

In [43]:
from sklearn.ensemble import RandomForestRegressor

In [41]:
X.shape[0] == y.shape[0]
X_train = X[:int(0.8*X.shape[0]),:]
y_train = y[:int(0.8*X.shape[0]),:]
X_test = X[int(0.8*X.shape[0]):,:]
y_test = y[int(0.8*X.shape[0]):,:]

In [53]:
regr = RandomForestRegressor(random_state=5, verbose=1)
regr.fit(X_train, y_train)

c:\users\laide\miniconda3\envs\xbt_aggregator\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=5, verbose=1, warm_start=False)

In [54]:
result = regr.predict(X_test)
regr.score(X_test,y_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


0.35530996661373226